In [170]:
import json
import os
import pandas as pd

# Define path
file_path = r"C:\Users\kyram\OneDrive\School\Prepossesed Data.json"

# Confirm the file exists
if not os.path.exists(file_path):
    print(f"The file does not exist at the specified path: {file_path}")
else:
    # Load the JSON 
    with open(file_path, 'r') as file:
        preprocessed_data = json.load(file)

    # Print the keys
    print(preprocessed_data.keys())

    # Access the key
    if 'public' in preprocessed_data:
        public_data = preprocessed_data['public']
        # Convert the dictionary back to a DataFrame
        df = pd.DataFrame.from_dict(public_data)
        print(df)
    else:
        print("'public' key not found in the JSON file")


dict_keys(['public'])
       Age   AsAm  AmerInd  AvgDrinksPerWeek  Black  DrinkDaysPerWeek  \
0     83.0  False      0.0               0.0    0.0               0.0   
1     58.0  False      0.0              35.0    0.0               7.0   
2     40.0  False      0.0               0.0    0.0               0.0   
3     66.0  False      0.0               0.0    0.0               0.0   
4     39.0  False      0.0               0.0    1.0               0.0   
...    ...    ...      ...               ...    ...               ...   
6247  48.0  False      0.0               4.0    0.0               2.0   
6248  50.0  False      0.0               0.0    0.0               0.0   
6249  55.0  False      0.0               3.0    1.0               1.0   
6250  62.0  False      0.0              84.0    0.0               7.0   
6251  57.0  False      0.0               0.0    0.0               0.0   

      DrinksOneOccasion  DrinksPerDay  Education  EverHadCancer  ...  \
0                   0.0      

In [171]:
# Import the necessary libraries
import numpy as np
import random
import sys
import tensorflow 

from collections import Counter

from imblearn.over_sampling import RandomOverSampler

from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [172]:
# Define X and y
X = df.copy().drop(['EverHadCancer', 'FamilyEverHadCancer2'], axis=1)

y = df['FamilyEverHadCancer2']

In [173]:
# Encode y
label_encoder = LabelEncoder()
y_numeric = label_encoder.fit_transform(y)

# Impute X
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)


In [174]:
# Scale X
numerical_transformer = StandardScaler()

X_imputed = numerical_transformer.fit_transform(X_imputed)


In [175]:
# Split into train and test splits
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y_numeric, test_size=0.2, random_state=42, stratify=y_numeric)

# Resample
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)


In [176]:
# Check shapes and distributions
print("Shape:", df.shape)
print("Dataset distribution:", Counter(df))

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("y_test Dataset distribution:", Counter(y_test))

print("X__train_resampled Shape", X_train_resampled.shape)
print("y_train_resampled Shape", y_train_resampled.shape)
print("y_train_resampled Dataset distribution:", Counter(y_train_resampled))

Shape: (6252, 59)
Dataset distribution: Counter({'Age': 1, 'AsAm': 1, 'AmerInd': 1, 'AvgDrinksPerWeek': 1, 'Black': 1, 'DrinkDaysPerWeek': 1, 'DrinksOneOccasion': 1, 'DrinksPerDay': 1, 'Education': 1, 'EverHadCancer': 1, 'FamilyEverHadCancer2': 1, 'FreqGoProvider': 1, 'HealthInsurance2': 1, 'HeardHPVVaccine2': 1, 'HPVCauseCancer_Cervical': 1, 'Hisp': 1, 'IncomeRanges': 1, 'MedConditions_Diabetes': 1, 'MedConditions_HighBP': 1, 'MedConditions_HeartCondition': 1, 'MedConditions_LungDisease': 1, 'OthPacIsl': 1, 'Smoke100': 1, 'Sunburned_Active': 1, 'Sunburned_Other': 1, 'Sunburned_Rec': 1, 'Sunburned_Work': 1, 'TimesSunburned': 1, 'UsedECigEver': 1, 'WhenPapTest': 1, 'White': 1, 'BirthGender_Female': 1, 'BirthGender_Male': 1, 'DelayNeededCare_I did not need any medical care in the past 12 months': 1, 'DelayNeededCare_No, I received the medical care I felt I needed': 1, 'DelayNeededCare_Yes': 1, 'DocTalkLDCT_Dont know': 1, 'DocTalkLDCT_I have never heard of this test': 1, 'DocTalkLDCT_No':

In [177]:
# Set seed
random.seed(42)

# Define model
model_1 = Sequential([
    Dense(57, activation='relu', input_shape=(57,)),  
    Dense(30, activation='relu'), 
    Dense(len(np.unique(y_numeric)), activation='softmax') 
])


# Compile model 
model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_1.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_1, accuracy_1 = model_1.evaluate(X_test, y_test)
print(f'Loss: {loss_1}, Accuracy: {accuracy_1}')

# Predict y
y_pred_1 = model_1.predict(X_test)
y_pred_classes_1 = np.argmax(y_pred_1, axis=1)

# F1 score and confusion matrix
f1_1 = f1_score(y_test, y_pred_classes_1, average='weighted')
conf_matrix_1 = confusion_matrix(y_test, y_pred_classes_1)

print(f'F1 Score: {f1_1}')
print('Confusion Matrix:')
print(conf_matrix_1)


Epoch 1/100
160/160 [==============================] - 2s 5ms/step - loss: 0.6160 - accuracy: 0.6736 - val_loss: 0.8405 - val_accuracy: 0.4053
Epoch 2/100
160/160 [==============================] - 0s 3ms/step - loss: 0.5609 - accuracy: 0.7104 - val_loss: 0.7999 - val_accuracy: 0.4546
Epoch 3/100
160/160 [==============================] - 0s 3ms/step - loss: 0.5417 - accuracy: 0.7233 - val_loss: 0.6613 - val_accuracy: 0.5352
Epoch 4/100
160/160 [==============================] - 0s 3ms/step - loss: 0.5257 - accuracy: 0.7315 - val_loss: 0.7687 - val_accuracy: 0.4812
Epoch 5/100
160/160 [==============================] - 0s 3ms/step - loss: 0.5115 - accuracy: 0.7446 - val_loss: 0.7970 - val_accuracy: 0.4397
Epoch 6/100
160/160 [==============================] - 0s 3ms/step - loss: 0.4981 - accuracy: 0.7511 - val_loss: 0.7602 - val_accuracy: 0.4679
Epoch 7/100
160/160 [==============================] - 0s 1ms/step - loss: 0.4839 - accuracy: 0.7605 - val_loss: 0.6650 - val_accuracy: 0.5673

In [178]:
# Set seed
random.seed(42)

# Define model
model_2 = Sequential([
    Dense(57, activation='relu', input_shape=(57,)),  
    Dense(30, activation='relu'), 
    Dense(15, activation='relu'), 
    Dense(len(np.unique(y_numeric)), activation='softmax') 
])


# Compile model 
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_2.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_2, accuracy_2 = model_2.evaluate(X_test, y_test)
print(f'Loss: {loss_2}, Accuracy: {accuracy_2}')

# Predict y
y_pred_2 = model_2.predict(X_test)
y_pred_classes_2 = np.argmax(y_pred_2, axis=1)

# F1 score and confusion matrix
f1_2 = f1_score(y_test, y_pred_classes_2, average='weighted')
conf_matrix_2 = confusion_matrix(y_test, y_pred_classes_2)

print(f'F1 Score: {f1_2}')
print('Confusion Matrix:')
print(conf_matrix_2)


Epoch 1/100
160/160 [==============================] - 2s 5ms/step - loss: 0.6347 - accuracy: 0.6581 - val_loss: 0.8107 - val_accuracy: 0.4225
Epoch 2/100
160/160 [==============================] - 1s 3ms/step - loss: 0.5705 - accuracy: 0.7153 - val_loss: 0.8477 - val_accuracy: 0.3850
Epoch 3/100
160/160 [==============================] - 1s 3ms/step - loss: 0.5442 - accuracy: 0.7241 - val_loss: 0.7258 - val_accuracy: 0.5047
Epoch 4/100
160/160 [==============================] - 1s 3ms/step - loss: 0.5235 - accuracy: 0.7413 - val_loss: 0.6376 - val_accuracy: 0.5704
Epoch 5/100
160/160 [==============================] - 1s 3ms/step - loss: 0.5080 - accuracy: 0.7519 - val_loss: 0.7279 - val_accuracy: 0.5000
Epoch 6/100
160/160 [==============================] - 1s 3ms/step - loss: 0.4873 - accuracy: 0.7703 - val_loss: 0.6050 - val_accuracy: 0.6135
Epoch 7/100
160/160 [==============================] - 1s 3ms/step - loss: 0.4701 - accuracy: 0.7824 - val_loss: 0.5581 - val_accuracy: 0.6659

In [179]:
# Set seed
random.seed(42)

# Define
model_3 = Sequential([
    Dense(57, activation='relu', input_shape=(57,)),  
    Dense(40, activation='relu'), 
    Dense(20, activation='relu'),
    Dense(10, activation='relu'), 
    Dense(len(np.unique(y_numeric)), activation='softmax') 
])


# Compile model 
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_3.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_3, accuracy_3 = model_3.evaluate(X_test, y_test)
print(f'Loss: {loss_3}, Accuracy: {accuracy_3}')

# Predict y
y_pred_3 = model_3.predict(X_test)
y_pred_classes_3 = np.argmax(y_pred_3, axis=1)

# F1 score and confusion matrix
f1_3 = f1_score(y_test, y_pred_classes_3, average='weighted')
conf_matrix_3 = confusion_matrix(y_test, y_pred_classes_3)

print(f'F1 Score: {f1_3}')
print('Confusion Matrix:')
print(conf_matrix_3)


Epoch 1/100
160/160 [==============================] - 2s 5ms/step - loss: 0.6364 - accuracy: 0.6421 - val_loss: 0.7549 - val_accuracy: 0.4014
Epoch 2/100
160/160 [==============================] - 1s 3ms/step - loss: 0.5602 - accuracy: 0.7155 - val_loss: 0.7878 - val_accuracy: 0.4178
Epoch 3/100
160/160 [==============================] - 1s 3ms/step - loss: 0.5399 - accuracy: 0.7276 - val_loss: 0.6945 - val_accuracy: 0.4765
Epoch 4/100
160/160 [==============================] - 1s 3ms/step - loss: 0.5224 - accuracy: 0.7413 - val_loss: 0.6782 - val_accuracy: 0.5196
Epoch 5/100
160/160 [==============================] - 1s 4ms/step - loss: 0.5048 - accuracy: 0.7476 - val_loss: 0.7939 - val_accuracy: 0.4249
Epoch 6/100
160/160 [==============================] - 1s 3ms/step - loss: 0.4874 - accuracy: 0.7654 - val_loss: 0.6766 - val_accuracy: 0.5250
Epoch 7/100
160/160 [==============================] - 1s 3ms/step - loss: 0.4716 - accuracy: 0.7753 - val_loss: 0.6852 - val_accuracy: 0.5407

In [180]:
# Set seed
random.seed(42)

# Define model
model_4 = Sequential([
    Dense(57, activation='relu', input_shape=(57,)),  
    Dense(40, activation='relu'), 
    Dense(30, activation='relu'), 
    Dense(20, activation='relu'),
    Dense(10, activation='relu'), 
    Dense(len(np.unique(y_numeric)), activation='softmax') 
])


# Compile model 
model_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit model 
model_4.fit(X_train_resampled, y_train_resampled, epochs=100, batch_size=32, validation_split=0.2)

# Evaluate model 
loss_4, accuracy_4 = model_4.evaluate(X_test, y_test)
print(f'Loss: {loss_4}, Accuracy: {accuracy_4}')

# Predict y
y_pred_4 = model_4.predict(X_test)
y_pred_classes_4 = np.argmax(y_pred_4, axis=1)

# F1 score and confusion matrix
f1_4 = f1_score(y_test, y_pred_classes_4, average='weighted')
conf_matrix_4 = confusion_matrix(y_test, y_pred_classes_4)

print(f'F1 Score: {f1_4}')
print('Confusion Matrix:')
print(conf_matrix_4)


Epoch 1/100
160/160 [==============================] - 3s 5ms/step - loss: 0.6030 - accuracy: 0.6855 - val_loss: 0.7825 - val_accuracy: 0.4444
Epoch 2/100
160/160 [==============================] - 1s 4ms/step - loss: 0.5599 - accuracy: 0.7192 - val_loss: 0.7601 - val_accuracy: 0.4460
Epoch 3/100
160/160 [==============================] - 1s 3ms/step - loss: 0.5416 - accuracy: 0.7344 - val_loss: 0.7915 - val_accuracy: 0.3967
Epoch 4/100
160/160 [==============================] - 0s 3ms/step - loss: 0.5257 - accuracy: 0.7403 - val_loss: 0.7934 - val_accuracy: 0.4507
Epoch 5/100
160/160 [==============================] - 1s 3ms/step - loss: 0.5083 - accuracy: 0.7544 - val_loss: 0.7825 - val_accuracy: 0.4851
Epoch 6/100
160/160 [==============================] - 1s 3ms/step - loss: 0.4884 - accuracy: 0.7679 - val_loss: 0.7305 - val_accuracy: 0.5070
Epoch 7/100
160/160 [==============================] - 1s 3ms/step - loss: 0.4705 - accuracy: 0.7810 - val_loss: 0.6488 - val_accuracy: 0.5869

In [181]:
# Compare the differnet versions of the model
print('Model_1: 1 layered model')
print(f'Loss: {loss_1}, Accuracy: {accuracy_1}')
print(f'F1 Score: {f1_1}')
print('Confusion Matrix:')
print(conf_matrix_1)

print('Model_2: 2 layered model')
print(f'Loss: {loss_2}, Accuracy: {accuracy_2}')
print(f'F1 Score: {f1_2}')
print('Confusion Matrix:')
print(conf_matrix_2)

print('Model_3: 3 layered model')
print(f'Loss: {loss_3}, Accuracy: {accuracy_3}')
print(f'F1 Score: {f1_3}')
print('Confusion Matrix:')
print(conf_matrix_3)

print('Model_4: 4 layered model')
print(f'Loss: {loss_4}, Accuracy: {accuracy_4}')
print(f'F1 Score: {f1_4}')
print('Confusion Matrix:')
print(conf_matrix_4)

Model_1: 1 layered model
Loss: 3.090261220932007, Accuracy: 0.6195043921470642
F1 Score: 0.6189449031724862
Confusion Matrix:
[[211 241]
 [235 564]]
Model_2: 2 layered model
Loss: 3.461904287338257, Accuracy: 0.6266986131668091
F1 Score: 0.6252850517124858
Confusion Matrix:
[[211 241]
 [226 573]]
Model_3: 3 layered model
Loss: 3.0062224864959717, Accuracy: 0.6418864727020264
F1 Score: 0.6381885169289934
Confusion Matrix:
[[209 243]
 [205 594]]
Model_4: 4 layered model
Loss: 2.802072286605835, Accuracy: 0.6442845463752747
F1 Score: 0.6431247931574672
Confusion Matrix:
[[223 229]
 [216 583]]


In [182]:
# Define the feature importance
weights = model_4.get_weights()

feature_importance = np.sum(np.abs(weights[0]), axis=1)

In [183]:
# Define the features
features = df.drop(['EverHadCancer', 'FamilyEverHadCancer2'], axis=1).columns

# Convert feature importance to a DataFrame
feature_importance_df = pd.DataFrame({
    'feature': features,
    'importance': feature_importance
})

# Sort by importance
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print(feature_importance_df)


                                              feature  importance
39                          DocTellColorectalTests_No   11.734979
0                                                 Age   11.071309
8                                           Education   11.061773
7                                        DrinksPerDay   10.848192
9                                      FreqGoProvider   10.625103
41                              QualityCare_Excellent   10.496998
1                                                AsAm   10.416945
31  DelayNeededCare_I did not need any medical car...   10.330906
5                                    DrinkDaysPerWeek   10.330372
40                         DocTellColorectalTests_Yes   10.305519
27                                        WhenPapTest   10.225266
15                             MedConditions_Diabetes   10.111240
49  SunEffectAfter1Hour_Burn mildly with some or n...   10.086460
10                                   HealthInsurance2   10.067721
47        

In [184]:
# Convert DataFrame to a dictionary
FNN_importance = {
    'public': feature_importance_df.to_dict(),  
}
# Create path
save_path = r"C:\Users\kyram\OneDrive\School\FNN2_importance.json"

# Confirm the dictionary exists
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# Save the data to a JSON file
with open(save_path, 'w') as file:
    json.dump(FNN_importance, file)

print(f"Data successfully saved to {save_path}")

Data successfully saved to C:\Users\kyram\OneDrive\School\FNN2_importance.json
